In [3]:
# inserting random numbers of objects, in random places in one image
import json
import random
import os
import math
from PIL import Image

def organize_points_clockwise(points):
    centroid = [sum(x[0] for x in points) / len(points), sum(x[1] for x in points) / len(points)]
    sorted_points = sorted(points, key=lambda p: math.atan2(p[1] - centroid[1], p[0] - centroid[0]))
    return sorted_points

def create_polygon_points(x, y, width, height):
    return organize_points_clockwise([
        [x, y],
        [x + width, y],
        [x + width, y + height],
        [x, y + height]
    ])

# Load the annotation file
with open('0A0WG84517.json', 'r') as file:
    annotation_data = json.load(file)

# Load the background image
background_image_path = annotation_data["imagePath"]
background_image = Image.open(background_image_path)

# Get Tote dimensions from the annotation
tote_points = annotation_data["shapes"][0]["points"]
tote_x_min = min(point[0] for point in tote_points)
tote_x_max = max(point[0] for point in tote_points)
tote_y_min = min(point[1] for point in tote_points)
tote_y_max = max(point[1] for point in tote_points)
tote_width = tote_x_max - tote_x_min
tote_height = tote_y_max - tote_y_min

# Set mean and standard deviation for the normal distribution
mean_objects = 5
std_dev_objects = 2
# Set an offset to move objects away from the edge
offset = 50

# Specify the path to your object images directory
object_images_directory = '/home/asma/Desktop/github/object insertion/out'

# List all files in the object images directory
object_images = [file for file in os.listdir(object_images_directory) if file.endswith('.jpg')]

# Generate a random number of objects based on a normal distribution
num_objects = round(random.gauss(mean_objects, std_dev_objects))

# Loop through each randomly selected object
for _ in range(num_objects):
    # Randomly pick an object from the list
    object_name = random.choice(object_images)

    # Load object image
    object_path = os.path.join(object_images_directory, object_name)
    object_image = Image.open(object_path)

    # Get random coordinates within the Tote with an offset
    x_position = random.randint(tote_x_min + offset, tote_x_max - offset - object_image.width)
    y_position = random.randint(tote_y_min + offset, tote_y_max - offset - object_image.height)

    # Paste the object on the background
    background_image.paste(object_image, (x_position, y_position))

    # Modify the annotation data for the new object
    new_object_annotation = {
        "label": "Object",
        "points": create_polygon_points(x_position, y_position, object_image.width, object_image.height),
        "group_id": None,
        "shape_type": "polygon",
        "flags": {}
    }

    # Append the new object annotation to the existing annotation data
    annotation_data["shapes"].append(new_object_annotation)

# Save the modified background image
background_image.save('output_image.jpg')

# Save the modified annotation data
with open('output_annotation.json', 'w') as output_file:
    json.dump(annotation_data, output_file, indent=2)
